<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/acme.jpeg width=100 align=left>

<img src="images/policy-workflow.png">

**Prerequisites:**
- Vault binary


Before you begin, execute the following command in a separate windows:

`VAULT_UI=true VAULT_REDIRECT_ADDR=http://127.0.0.1:8200 vault server -log-level=trace -dev -dev-root-token-id=root -dev-listen-address=127.0.0.1:8200 -dev-ha -dev-transactional`


In [49]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true
export VAULT_ROOT_TOKEN=root

In [48]:
vault --version

Vault v1.6.0 (7ce0bd9691998e0443bc77e98b1e2a4ab1e965d4)


In [47]:
vault status

Key             Value
---             -----
Seal Type       shamir
Initialized     true
Sealed          false
Total Shares    3
Threshold       2
Version         1.6.0
Storage Type    inmem_transactional_ha
Cluster Name    vault-cluster-eb5fbb0c
Cluster ID      ee93a68b-c56b-0275-1460-88c0df32a576
HA Enabled      true
HA Cluster      https://127.0.0.1:8201
HA Mode         active


#### Note the Initialized and Sealed status above.

## Create Policy
**No policy means no permission**  

ACL policy path supports wildcard matching ("+", "*")

List of capabilities  

| Capability | HTTP Verbs |
|------------|------------|
| create | POST/PUT |
| read | GET |
| update | POST/PUT |
| delete | DELETE |
| list | LIST |
| sudo | allows access to paths that are **root-protected** |
| deny | disallows access regardless of other capabilities |

Two out-of-the-box policies:
* **root** policy - superuser with all permissions
* **default** policy - common permissions  


In [50]:
# Work with kv secrets engine

cat << EOF > /tmp/base.hcl

path "kv/data/training_*" {
  capabilities = [ "create", "read" ]
}

path "kv/data/+/apikey" {
  capabilities = [ "create", "read", "update", "delete" ]
}

EOF


**Note:** in *key/value secret engine 2*, the path to write policy would be **kv/data/\<path\>** even though the CLI command to the path is **kv/\<path\>**

In [51]:
vault policy write base /tmp/base.hcl

Success! Uploaded policy: base


In [61]:
vault secrets list

Error listing secrets engines: Error making API request.

URL: GET http://127.0.0.1:8200/v1/sys/mounts
Code: 403. Errors:

* 1 error occurred:
	* permission denied




In [56]:
vault policy read base

path "kv/data/training_*" {
  capabilities = [ "create", "read" ]
}

path "kv/data/+/apikey" {
  capabilities = [ "create", "read", "update", "delete" ]
}


In [62]:
vault secrets enable -path=kv/ kv-v2

Success! Enabled the kv-v2 secrets engine at: kv/


#### Create a token attached to the newly *base* policy

In [57]:
vault token create -policy="base"

Key                  Value
---                  -----
token                s.7XdKUkHaNUMeWkgYu0mgyaPo
token_accessor       5QmCEYkbHQb0BxjSySRNVAU7
token_duration       768h
token_renewable      true
token_policies       ["base" "default"]
identity_policies    []
policies             ["base" "default"]


**Note:** every token automatically gets *default* policy attached.

In [80]:
vault login "s.7XdKUkHaNUMeWkgYu0mgyaPo"

Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                  Value
---                  -----
token                s.7XdKUkHaNUMeWkgYu0mgyaPo
token_accessor       5QmCEYkbHQb0BxjSySRNVAU7
token_duration       767h15m53s
token_renewable      true
token_policies       ["base" "default"]
identity_policies    []
policies             ["base" "default"]


---
## Test "base" Policy

Using base token, you have very limited permissions.

In [59]:
vault policy list

Error listing policies: Error making API request.

URL: GET http://127.0.0.1:8200/v1/sys/policies/acl?list=true
Code: 403. Errors:

* 1 error occurred:
	* permission denied




Using base token, you have very limited permissions.  THe base policy does not have a rule on `sys/policy` path.  Lack of policy means **no permission** on that path.  Therefore, "permission denied" error is the expected behavior.

In [63]:
vault kv put kv/training_test password="p@ssw0rd"

Key              Value
---              -----
created_time     2020-12-24T12:33:00.085Z
deletion_time    n/a
destroyed        false
version          1


#### Read the data back

In [64]:
vault kv get kv/training_test

====== Metadata ======
Key              Value
---              -----
created_time     2020-12-24T12:33:00.085Z
deletion_time    n/a
destroyed        false
version          1

====== Data ======
Key         Value
---         -----
password    p@ssw0rd


#### Base policy only granst "create" and "read".  With the absence of "update" permission, this operation fails.

In [67]:
vault kv put kv/training_test password="password1234"

Error writing data to kv/data/training_test: Error making API request.

URL: PUT http://127.0.0.1:8200/v1/kv/data/training_test
Code: 403. Errors:

* 1 error occurred:
	* permission denied




In [68]:
vault kv put kv/team-eng/apikey api_key="123455"

Key              Value
---              -----
created_time     2020-12-24T12:36:33.754538Z
deletion_time    n/a
destroyed        false
version          1


In [69]:
vault kv delete kv/team-eng/apikey

Success! Data deleted (if it existed) at: kv/team-eng/apikey


In [70]:
vault kv put kv/training_ year="2020"

Key              Value
---              -----
created_time     2020-12-24T12:37:12.846247Z
deletion_time    n/a
destroyed        false
version          1


&nbsp;   
### Check Token capabilities
The `vault token capabilities` command fetches the capabilities of a token for a given path.  You can review the policy, but if your token has multiple policies attached, you have to review all the associated policies which can get troublesome.

In [81]:
vault token capabilities kv/data/training_dev

create, read


In [82]:
vault token capabilities kv/data/splunk/apikey

create, delete, read, update


#### Try another path that is not permitted by the `base` policy

In [83]:
vault token capabilities kv/data/test

deny


In [85]:
vault login root >/dev/null
vault token capabilities kv/data/test

root


---
### Policy for Policies
To allow users to be able to manage policies, you need to write policies for that

In [77]:
cat << EOF > /tmp/manage-policies.hcl

path "sys/policies/acl/*" {
  capabilities = ["create", "read", "update", "delete", "list", "sudo"]
}
# Create and manage ACL policies via API
path "sys/policies/acl" {
  capabilities = ["list"]
}
EOF

&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">